# Experiment 012: Lattice/Crystalline Packing for Large N

Based on the Medium article insight: top teams use HYBRID strategy:
- N < 58: SA for chaotic packings
- N > 58: Crystalline/Lattice packing

This experiment implements lattice-based packing for large N values (150-200)
which contribute ~50% of total score.

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import random
import time

getcontext().prec = 25
scale_factor = Decimal("1e15")

print("Libraries loaded")

In [ ]:
class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def get_params(self):
        return self.center_x, self.center_y, self.angle

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

In [ ]:
def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                return True
    return False

def load_trees(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"]).lstrip('s')
        y = str(row["y"]).lstrip('s')
        deg = str(row["deg"]).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    return trees

print("Helper functions defined")

In [ ]:
def create_hexagonal_lattice(n, spacing=0.5):
    """Create a hexagonal lattice arrangement for N trees.
    
    Hexagonal packing is the most efficient for circles.
    For trees, we alternate between 0 and 180 degree rotations.
    """
    trees = []
    
    # Calculate grid dimensions
    # For hexagonal packing, rows are offset by half the spacing
    cols = int(np.ceil(np.sqrt(n * 1.2)))  # Slightly more columns than rows
    rows = int(np.ceil(n / cols))
    
    row_height = spacing * np.sqrt(3) / 2  # Hexagonal row spacing
    
    count = 0
    for row in range(rows):
        for col in range(cols):
            if count >= n:
                break
            
            # Hexagonal offset for odd rows
            x_offset = (spacing / 2) if row % 2 == 1 else 0
            
            x = col * spacing + x_offset
            y = row * row_height
            
            # Alternate angles for interlocking
            angle = 0 if (row + col) % 2 == 0 else 180
            
            trees.append(ChristmasTree(str(x), str(y), str(angle)))
            count += 1
        
        if count >= n:
            break
    
    return trees

# Test hexagonal lattice
test_trees = create_hexagonal_lattice(100, spacing=0.6)
print(f"Created {len(test_trees)} trees")
print(f"Score: {calculate_score(test_trees):.6f}")
print(f"Has collision: {has_collision(test_trees)}")

In [ ]:
def create_interlocking_grid(n, dx=0.4, dy=0.5):
    """Create an interlocking grid where trees with 0 and 180 degree rotations
    are placed to interlock their branches.
    
    This is similar to the egortrushin tessellation approach.
    """
    trees = []
    
    # Calculate grid dimensions
    # We need n trees total, arranged in a grid with alternating orientations
    cols = int(np.ceil(np.sqrt(n)))
    rows = int(np.ceil(n / cols))
    
    count = 0
    for row in range(rows):
        for col in range(cols):
            if count >= n:
                break
            
            x = col * dx
            y = row * dy
            
            # Alternate angles in a checkerboard pattern
            angle = 0 if (row + col) % 2 == 0 else 180
            
            trees.append(ChristmasTree(str(x), str(y), str(angle)))
            count += 1
        
        if count >= n:
            break
    
    return trees

# Test interlocking grid
test_trees = create_interlocking_grid(100, dx=0.5, dy=0.6)
print(f"Created {len(test_trees)} trees")
print(f"Score: {calculate_score(test_trees):.6f}")
print(f"Has collision: {has_collision(test_trees)}")

In [ ]:
def optimize_lattice_spacing(n, create_func, dx_range=(0.3, 0.8), dy_range=(0.3, 0.8), steps=20):
    """Find optimal spacing for a lattice configuration."""
    best_score = float('inf')
    best_trees = None
    best_params = None
    
    for dx in np.linspace(dx_range[0], dx_range[1], steps):
        for dy in np.linspace(dy_range[0], dy_range[1], steps):
            trees = create_func(n, dx=dx, dy=dy)
            
            if not has_collision(trees):
                score = calculate_score(trees)
                if score < best_score:
                    best_score = score
                    best_trees = trees
                    best_params = (dx, dy)
    
    return best_score, best_trees, best_params

# Load current best for comparison
current_best_df = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')

# Test on N=100
print("Optimizing lattice spacing for N=100...")
current_score_100 = calculate_score(load_trees(100, current_best_df))
print(f"Current best N=100: {current_score_100:.6f}")

lattice_score, lattice_trees, params = optimize_lattice_spacing(100, create_interlocking_grid)
if lattice_trees:
    print(f"Lattice N=100: {lattice_score:.6f} (dx={params[0]:.3f}, dy={params[1]:.3f})")
    print(f"Improvement: {current_score_100 - lattice_score:.6f}")
else:
    print("No valid lattice configuration found")

In [ ]:
# The simple lattice approach is not finding improvements
# Let me try the egortrushin approach more carefully:
# Create a tessellation with 2 layers (0 and 180 degree trees)

def create_tessellation_v2(n, dx=0.4, dy=0.5, offset_x=0.2, offset_y=0.25):
    """Create tessellation with two interlocking layers.
    
    Layer 1: Trees at angle 0, on a regular grid
    Layer 2: Trees at angle 180, offset from layer 1
    
    This matches the egortrushin approach more closely.
    """
    trees = []
    
    # Calculate how many trees per layer
    n_per_layer = n // 2
    n_layer1 = n_per_layer + (n % 2)  # Extra tree goes to layer 1
    n_layer2 = n_per_layer
    
    # Calculate grid dimensions for each layer
    cols = int(np.ceil(np.sqrt(n_per_layer * 1.5)))
    rows = int(np.ceil(n_per_layer / cols))
    
    # Layer 1: angle = 0
    count = 0
    for row in range(rows + 1):
        for col in range(cols + 1):
            if count >= n_layer1:
                break
            x = col * dx
            y = row * dy
            trees.append(ChristmasTree(str(x), str(y), "0"))
            count += 1
        if count >= n_layer1:
            break
    
    # Layer 2: angle = 180, offset
    count = 0
    for row in range(rows + 1):
        for col in range(cols + 1):
            if count >= n_layer2:
                break
            x = col * dx + offset_x
            y = row * dy + offset_y
            trees.append(ChristmasTree(str(x), str(y), "180"))
            count += 1
        if count >= n_layer2:
            break
    
    return trees[:n]  # Ensure exactly n trees

# Test tessellation v2
test_trees = create_tessellation_v2(100, dx=0.5, dy=0.6, offset_x=0.25, offset_y=0.3)
print(f"Created {len(test_trees)} trees")
print(f"Score: {calculate_score(test_trees):.6f}")
print(f"Has collision: {has_collision(test_trees)}")

In [ ]:
def optimize_tessellation_v2(n, dx_range=(0.35, 0.6), dy_range=(0.4, 0.7), steps=15):
    """Find optimal parameters for tessellation v2."""
    best_score = float('inf')
    best_trees = None
    best_params = None
    
    for dx in np.linspace(dx_range[0], dx_range[1], steps):
        for dy in np.linspace(dy_range[0], dy_range[1], steps):
            # Offset is typically half the spacing
            for offset_ratio in [0.4, 0.45, 0.5, 0.55, 0.6]:
                offset_x = dx * offset_ratio
                offset_y = dy * offset_ratio
                
                trees = create_tessellation_v2(n, dx=dx, dy=dy, offset_x=offset_x, offset_y=offset_y)
                
                if not has_collision(trees):
                    score = calculate_score(trees)
                    if score < best_score:
                        best_score = score
                        best_trees = [t.clone() for t in trees]
                        best_params = (dx, dy, offset_x, offset_y)
    
    return best_score, best_trees, best_params

# Test on N=200 (largest, contributes most to score)
print("Optimizing tessellation for N=200...")
current_score_200 = calculate_score(load_trees(200, current_best_df))
print(f"Current best N=200: {current_score_200:.6f}")

tess_score, tess_trees, params = optimize_tessellation_v2(200)
if tess_trees:
    print(f"Tessellation N=200: {tess_score:.6f}")
    print(f"Params: dx={params[0]:.3f}, dy={params[1]:.3f}, ox={params[2]:.3f}, oy={params[3]:.3f}")
    print(f"Improvement: {current_score_200 - tess_score:.6f}")
else:
    print("No valid tessellation configuration found")

In [ ]:
# The tessellation approach is also not finding improvements
# The current solutions are already very well optimized

# Let me check the current best solutions for large N to understand their structure
print("Analyzing current best solutions for large N:")
print("="*60)

for n in [150, 175, 200]:
    trees = load_trees(n, current_best_df)
    score = calculate_score(trees)
    
    # Analyze angle distribution
    angles = [float(t.angle) for t in trees]
    angle_counts = {}
    for a in angles:
        a_rounded = round(a / 10) * 10  # Round to nearest 10 degrees
        angle_counts[a_rounded] = angle_counts.get(a_rounded, 0) + 1
    
    # Get bounding box
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    
    print(f"\nN={n}: score={score:.6f}")
    print(f"  Bounding box: ({min_x:.3f}, {min_y:.3f}) to ({max_x:.3f}, {max_y:.3f})")
    print(f"  Size: {max_x - min_x:.3f} x {max_y - min_y:.3f}")
    print(f"  Top 5 angles: {sorted(angle_counts.items(), key=lambda x: -x[1])[:5]}")

In [ ]:
# The current solutions use a variety of angles, not just 0/180
# This suggests they're using more sophisticated optimization

# Let me try a different approach: SA optimization starting from the current best
# but with MUCH more aggressive parameters

def aggressive_sa(trees, nsteps=50000, Tmax=1.0, Tmin=0.00001, position_delta=0.02, angle_delta=15):
    """Run aggressive SA with many steps and high temperature."""
    best_trees = [t.clone() for t in trees]
    best_score = calculate_score(best_trees)
    
    current_trees = [t.clone() for t in trees]
    current_score = best_score
    
    T = Tmax
    cooling_rate = (Tmin / Tmax) ** (1.0 / nsteps)
    
    improvements = 0
    
    for step in range(nsteps):
        # Pick a random tree
        idx = random.randint(0, len(current_trees) - 1)
        tree = current_trees[idx]
        
        old_x, old_y, old_angle = tree.get_params()
        
        # Perturb
        new_x = old_x + Decimal(str(random.uniform(-position_delta, position_delta)))
        new_y = old_y + Decimal(str(random.uniform(-position_delta, position_delta)))
        new_angle = (old_angle + Decimal(str(random.uniform(-angle_delta, angle_delta)))) % 360
        
        tree.center_x = new_x
        tree.center_y = new_y
        tree.angle = new_angle
        tree.__init__(str(new_x), str(new_y), str(new_angle))
        
        # Check collision
        if has_collision(current_trees):
            tree.__init__(str(old_x), str(old_y), str(old_angle))
            T *= cooling_rate
            continue
        
        new_score = calculate_score(current_trees)
        
        # Accept or reject
        delta = new_score - current_score
        if delta < 0 or random.random() < np.exp(-delta / T):
            current_score = new_score
            if current_score < best_score:
                best_score = current_score
                best_trees = [t.clone() for t in current_trees]
                improvements += 1
        else:
            tree.__init__(str(old_x), str(old_y), str(old_angle))
        
        T *= cooling_rate
        
        if step % 10000 == 0:
            print(f"Step {step}: best={best_score:.6f}, T={T:.6f}, improvements={improvements}")
    
    return best_score, best_trees, improvements

print("Aggressive SA defined")

In [ ]:
# Test aggressive SA on N=200
print("Running aggressive SA on N=200...")
random.seed(42)

trees_200 = load_trees(200, current_best_df)
current_score_200 = calculate_score(trees_200)
print(f"Current score: {current_score_200:.6f}")

start_time = time.time()
new_score, new_trees, improvements = aggressive_sa(trees_200, nsteps=30000)
elapsed = time.time() - start_time

print(f"\nFinal score: {new_score:.6f}")
print(f"Improvement: {current_score_200 - new_score:.6f}")
print(f"Time: {elapsed:.1f}s")
print(f"Total improvements: {improvements}")

In [ ]:
# Still no improvement! The current solutions are at extremely strong local optima.

# Let me calculate the total score and see how close we are to target
print("Calculating total score...")
total_score = 0
for n in range(1, 201):
    trees = load_trees(n, current_best_df)
    total_score += calculate_score(trees)

print(f"Total score: {total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)")

# The gap is 1.71 points (2.42%)
# This is a SIGNIFICANT gap that cannot be closed by local optimization

In [ ]:
# Summary and save metrics
print("\nSummary of Experiment 012:")
print("="*60)
print(f"Current best total score: {total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {total_score - 68.919154:.6f} (2.42%)")
print()
print("Approaches tried:")
print("1. Hexagonal lattice packing: WORSE than current")
print("2. Interlocking grid: WORSE than current")
print("3. Tessellation v2 (egortrushin-style): WORSE than current")
print("4. Aggressive SA on N=200: NO improvement")
print()
print("Conclusion: Current solutions are at extremely strong local optima.")
print("Lattice/crystalline approaches produce WORSE configurations.")
print("The current solutions use sophisticated angle optimization that")
print("simple lattice patterns cannot match.")

In [ ]:
# Save metrics
import json

metrics = {
    'cv_score': total_score,
    'target': 68.919154,
    'gap': total_score - 68.919154,
    'approaches_tried': [
        'hexagonal_lattice',
        'interlocking_grid',
        'tessellation_v2',
        'aggressive_sa_n200'
    ],
    'result': 'no_improvement',
    'conclusion': 'Lattice approaches produce worse configurations than current solutions'
}

with open('/home/code/experiments/012_lattice_packing/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Metrics saved")
print(f"CV score: {metrics['cv_score']:.6f}")